In [2]:
import ipykernel
ipykernel.__version__
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')  

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/JanayeCheong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
cxr_ids = pd.read_csv('../../data/cxr_ids(in).csv')

cxr_ids.drop_duplicates
patient_ids = cxr_ids['subject_id'].to_list()
study_ids = cxr_ids['study_id'].to_list()
dicom_id = cxr_ids['dicom_id'].to_list()

study_ids
patient_ids
dicom_id

['1f239460-e00a31a5-81bdb260-f2929be7-f7cb2f7d',
 '82d06e5f-4f17e47a-3a96a851-d9454252-862ff5be',
 'fc215cad-d1060eb4-32e74e2e-15e2c40b-d01e66db',
 '9b1832b0-9cf4f257-fd4bbb42-06957578-0bf69814',
 '120b1593-d73e54d8-6d999909-ce1d93ce-d3b26f46',
 '410a98c3-ef15f3d3-fc27663b-f45190b1-4ce4c617',
 'f8929db5-e4ca050e-7c79a8a4-1fc58b26-08c88391',
 '95ba69a3-378811ac-5a17e8b9-0cfd0582-6260d469',
 '6e58e826-fc08e4ee-a9ff0245-34c6f4cd-cd0aedc8',
 '8f2ba1e5-0bb91210-84ad4a2f-eebe2f6b-34444ab2',
 '8765253d-3864f87b-412bbbd0-d6a2a59f-84404f13',
 '44a44e52-6e7ffefa-3de7d34b-1e4ced24-5ab1a492',
 '5ce74c55-cbd4a185-29acfad5-4a12e697-0857b07a',
 '254e3ab4-229a5808-099b2630-0b9e143a-e36b72bb',
 '7744e910-ffab52cd-090a9e79-e48e757c-c35585fa',
 '75ca2461-889ad0b2-9cde532f-d87905a5-56e2e55c',
 '960b7dbb-35d3301a-a26dcde1-bde7c504-f48f6369',
 '2e9aecc6-4396045c-f38e49ed-f1ffbbb7-f318f280',
 '6002eac9-ef518a7c-23ed107f-93447158-28e648fb',
 'cbab4e4c-5c0773d9-8a685bb7-10444fd3-a4ec57b0',
 'd5f03c44-e3b7efe0-

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from pathlib import Path

def parse_tfrecord(raw_tf):
    """
    Parse a single TFRecord file and extract the embedding values.

    Parameters:
        raw_tf: Path to the .tfrecord file

    Returns:
        A numpy array of embedding values
    """
    raw_dataset = tf.data.TFRecordDataset([raw_tf])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value

    return np.array(embedding_values)

def process_study_embeddings(cxr_ids, base_path):
    """
    Process embeddings from folders organized by study_id.

    Parameters:
        cxr_ids: DataFrame containing the study_id and subject_id mappings
        base_path: Root directory containing folders named by study_id
        
    Returns:
        A pandas DataFrame containing study_id, subject_id, and embeddings.
    """
    all_embeddings = []

    for _, row in cxr_ids.iterrows():
        subject_id = row['subject_id']
        study_id = row['study_id']
        dicom_id = row['dicom_id']

        folder_prefix = f"p{str(subject_id)[:2]}"
        folder_path = Path(base_path) / folder_prefix / f"p{subject_id}" / f"s{study_id}"
        tfrecord_files = list(folder_path.glob("*.tfrecord"))

        if not tfrecord_files:
            print(f"Warning: No TFRecord file found in folder {folder_path}")
            continue

        try:
            # Assuming one TFRecord file per folder
            tfrecord_path = tfrecord_files[0]
            embedding = parse_tfrecord(str(tfrecord_path))

            # Store the result
            all_embeddings.append({
                'study_id': study_id,
                'subject_id': subject_id,
                'dicom_id': dicom_id,
                'embedding': embedding
            })
        except Exception as e:
            print(f"Error processing folder {folder_path}: {e}")
            continue

    # Convert to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)
    return embeddings_df

def save_embeddings_to_csv(df, output_path):
    """
    Save the embeddings DataFrame to a CSV file.
    
    Parameters:
        df: DataFrame containing embeddings
        output_path: File path for the output CSV
    """
    # Convert embeddings to list for CSV compatibility
    df['embedding'] = df['embedding'].apply(lambda x: x.tolist())
    df.to_csv(output_path, index=False)
    print(f"Embeddings saved to {output_path}")

# Example usage
if __name__ == "__main__":

    base_path = '../../data/image_files'  # Path to the root directory containing study folders
    output_csv_path = 'image_embeddings.csv'

    print("Processing image embeddings...")
    embeddings_df = process_study_embeddings(cxr_ids, base_path)

    print("Saving embeddings to CSV...")
    save_embeddings_to_csv(embeddings_df, output_csv_path)

    print("\nSample of the processed embeddings:")
    print(embeddings_df.head())

Processing image embeddings...


2024-12-07 19:51:20.470198: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-07 19:51:39.398061: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Saving embeddings to CSV...
Embeddings saved to study_embeddings.csv

Sample of the processed embeddings:
   study_id  subject_id                                      dicom_id  \
0  57106576    18110461  1f239460-e00a31a5-81bdb260-f2929be7-f7cb2f7d   
1  52444794    15447063  82d06e5f-4f17e47a-3a96a851-d9454252-862ff5be   
2  58791719    13243522  fc215cad-d1060eb4-32e74e2e-15e2c40b-d01e66db   
3  51779043    11423061  9b1832b0-9cf4f257-fd4bbb42-06957578-0bf69814   
4  58785779    15379716  120b1593-d73e54d8-6d999909-ce1d93ce-d3b26f46   

                                           embedding  
0  [-1.0478514432907104, -0.9893836975097656, 0.9...  
1  [0.14750519394874573, -0.8777117133140564, 1.2...  
2  [-0.6212253570556641, -0.6505540609359741, 1.1...  
3  [-0.6631452441215515, -0.5168229341506958, 1.0...  
4  [-0.08448687195777893, -1.0461037158966064, 1....  
